# Projeto Final - Análise de Gols em uma Partida de Futebol
Autor: Paulo Victor Lima |   Orientador : Sergio Lima Netto |    Universidade Federal do Rio de Janeiro - Escola Politécnica - Departamente de Engenharia Eletrônica e de Computação

## Importação das Bibliotecas

In [ ]:
#Instalação das bibliotecas

!pip install pandas
!pip install numpy
!pip install seaborn
!pip install matplotlib.pyplot
!pip install plotly.express
!pip install glob

In [1]:
#Importação das bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import os
from sklearn.ensemble import RandomForestClassifier

## Importação Base de Dados

In [11]:
def set_config():
    from numpy import loadtxt

    #Carrega as features a serem consideradas
    features_types = []
    features_types = loadtxt("..\config\Features_Types.dat", comments="#", delimiter='\n', dtype=str, ndmin=1)

    ## Caminho padrão das partidas
    match_generic_path = '..\\database\\**\\'

    # Nome Padrão arquivo "Highlights"
    highlights_file_name = 'highlights2.csv'

Definição das Funções de leitura da base de dados

In [18]:
def import_moments_classifictaion(match, len):
    
    # Nome Padrão arquivo "Highlights"
    highlights_file_name = 'highlights.csv'

    #Caminho para os arquivos de features
    match_by_feature_path = match + highlights_file_name

    #Data frame com todos os intervalos de highligths de uma partida
    highlights_file =  pd.DataFrame()

    #Data frame com os intervalos de highligths de uma partida
    highlights_of_match = pd.DataFrame(list(range(1,len)), columns=['classification'], index=list(range(1,len)))

    #nome das colunas do frame de highlights
    colnames=['first_frame','last_frame', 'classification'] 

    #Data Frame com todos os highlights de uma partida
    highlights_file = pd.read_csv(match_by_feature_path, names=colnames, index_col=None, sep =',', header=None)

    highlights_of_match['classification'] = np.nan
    
    for index,row in highlights_file.iterrows():        
        first_frame = int(row['first_frame']) if int(row['first_frame']) != ' ' else 0
        last_frame = int(row['last_frame']) if row['last_frame'] != ' ' else int(row['first_frame'])
        classification = row['classification']
        highlights_of_match.loc[first_frame:last_frame, 'classification'] = classification
        
    return highlights_of_match['classification']


In [28]:
def import_database ():

    from numpy import loadtxt

    #Carrega as features a serem consideradas
    features_types = []
    features_types = loadtxt("..\config\Features_Types.dat", comments="#", delimiter='\n', dtype=str, ndmin=1)

    ## Caminho padrão das partidas
    match_generic_path = '..\\database\\Treinamento\\**\\'

    #Lista de partidas existentes na base
    all_matches = glob.glob(match_generic_path, recursive = False)

    ## Data Frame com todos os momementos de todas as partidas
    all_moments = pd.DataFrame()
    ## Array de Data Frames com todos os Data Frames das partidas
    all_moments_array = []

    for match in all_matches:
        print(match)

        #Data frame com todos os momentos de uma partida
        moments_of_match=  pd.DataFrame()

        for feature in features_types:

            #Caminho para os arquivos de features
            match_by_feature_path = match + feature

            #Data Frame com todos os momentos de uma partida de uma feature especifica
            moments_of_match[feature] = pd.read_csv(match_by_feature_path, index_col=None, sep ='\n', thousands=r".", header=None)
        
        #Preenche a coluna 'Match' com a partida
        moments_of_match['Match'] = [match_by_feature_path.split('\\')[-2] for _ in range(len(moments_of_match[feature]))]

            
        moments_of_match['Classification'] = import_moments_classifictaion(match, len(moments_of_match[feature]))

        moments_of_match.dropna(axis=0, how='any',inplace=True)

        #Adiciona ao array de Data Frames o Data Frame gerado para a partida
        all_moments_array.append(moments_of_match)

    all_moments = pd.concat(all_moments_array,  sort=False, axis=0, ignore_index=True)


    all_moments.to_csv("moments.csv", index=False, sep=';')

    return all_moments

    

Função Principal

In [29]:
# Configurações Gerais
# set_config()
# Importa a base de dados
all_moments = pd.DataFrame()

all_moments = import_database()

random_forest = RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0)
random_forest.fit()





..\database\Treinamento\amg_vas\
..\database\Treinamento\arg_ger\
..\database\Treinamento\arg_mex\
..\database\Treinamento\arg_nig\
..\database\Treinamento\arg_sko\
..\database\Treinamento\bar_int\
..\database\Treinamento\bay_int\
..\database\Treinamento\bra_chi\
..\database\Treinamento\bra_ita\
..\database\Treinamento\bra_ned\
